In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch
from torch.autograd import Variable
from torch import nn, optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import operator

In [0]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))])

In [0]:
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# flow_from_directory => Keras
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

In [0]:
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
# flow_from_directory => Keras
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [0]:
class Full_Connected_Net(nn.Module):
    def __init__(self):
        super(Full_Connected_Net, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(1*28*28, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, 10))
        
        
    def forward(self, x):
        x = x.view(-1, 1*28*28)
        x = self.classifier(x)
        return x

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.classifier = nn.Sequential(
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, 10))
        
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 64*7*7)
        x = self.classifier(x)
        return x

In [0]:
net = Net()
if torch.cuda.is_available():
  net = net.cuda()

In [0]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

In [0]:
import time

for epoch in range(2):
    start = time.time()
    for i, data in enumerate(trainloader):
        inputs, labels = data
        
        if torch.cuda.is_available():
          inputs = Variable(inputs).cuda()
          labels = Variable(labels).cuda()
        else:
          inputs = Variable(inputs)
          labels = Variable(labels)
        
        # forward + backward + optimize
        
        # zeroes the gradient buffers of all parameters
        optimizer.zero_grad()
        outputs = net(inputs)
        # calculate the loss
        loss = loss_function(outputs, labels)
        # backpropagation
        loss.backward()
        # Does the update after calculating the gradients
        optimizer.step()
        
        if (i+1) % 500 == 0: # print every 100 mini-batches
            print('[%d, %5d] loss: %.4f' % (epoch, i+1, loss.data[0]))
    print ('Time taken {}'.format(time.time() - start))

In [0]:
s = time.time()
c = 0
for data in trainloader:
  c+=1
print (time.time()-s)